# VANGUARD AB TEST


## METADATA HELP

This comprehensive set of fields will guide your analysis, helping you unravel the intricacies of client behavior and preferences.

- **client_id**: Every client’s unique ID.
- **variation**: Indicates if a client was part of the experiment.
- **visitor_id**: A unique ID for each client-device combination.
- **visit_id**: A unique ID for each web visit/session.
- **process_step**: Marks each step in the digital process.
- **date_time**: Timestamp of each web activity.
- **clnt_tenure_yr**: Represents how long the client has been with Vanguard, measured in years.
- **clnt_tenure_mnth**: Further breaks down the client’s tenure with Vanguard in months.
- **clnt_age**: Indicates the age of the client.
- **gendr**: Specifies the client’s gender.
- **num_accts**: Denotes the number of accounts the client holds with Vanguard.
- **bal**: Gives the total balance spread across all accounts for a particular client.
- **calls_6_mnth**: Records the number of times the client reached out over a call in the past six months.
- **logons_6_mnth**: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.


In [ ]:
%load_ext autoreload
%autoreload 2 

In [634]:
import pandas as pd
from cleaning import *
from mining import *
from dotenv import load_dotenv
import os
import yaml

In [ ]:
# Load environment variables
load_dotenv()

In [ ]:
# Load config
config = parse_config()

## Data Mining

In [637]:
#TODO: find a way to import and merge web data 2
#TODO: adapt function to remote url + save to sql to prevent large repo


In [638]:
dataFrames = { name:import_data_from_config(config, name) for name in config['tables']}

In [ ]:
display_dataFrames(dataFrames)

## Data Cleaning

In [640]:
#TODO: don't impose categories?
#TODO: consider binning / pd.cut / qcut for numerical data

In [641]:
# Rename columns
dataFrames = rename_columns(dataFrames, config)

In [ ]:
# select columns
dataFrames = select_columns(dataFrames, config)
display_dataFrames(dataFrames)

In [643]:
# Data formatting

In [644]:
# Data Typing

In [645]:
# Handle duplicates

In [646]:
# Handle missing values

## Data Exploration

In [647]:
# Handle outliers

In [648]:
#frequency tables

## Analysis

## Visualizations

## Conclusions